In [0]:


import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY, PG_AA_HOSTS, PG_AA_NAME,PG_AA_ACCESS_ID,PG_AA_SECRET_KEY

PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)

PG_AA_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


begin_date = datetime(2013, 1, 1)
end_date = datetime(2014, 12, 31)

DATE_GRANULARITY_MAPPINGLIST = {
    "daily": get_date_list(begin_date, end_date, "D"),
    "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    "monthly": get_date_list(begin_date, end_date, "M")
    }


# DATE_GRANULARITY_MAPPINGLIST["monthly"].reverse()
# DATE_GRANULARITY_MAPPINGLIST["weekly"].reverse()
# DATE_GRANULARITY_MAPPINGLIST["daily"].reverse()

In [0]:

basic_dump="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_app.v2/fact/granularity=monthly/"
basic="s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=daily/"
segment_by_age_gender_sql="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v2/fact/granularity={granularity}/month={date}/"
segment_by_audience="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ad_app.v2/fact/granularity={granularity}/month={date}/"
segment_by_product_sql="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v2/fact/granularity={granularity}/month={date}"
app_cross_app="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ca_app.v2/fact/granularity={granularity}/month={date}/"
app_retention="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/fact/granularity=monthly/"
start_str="2013-01"
end_str="2014-12"

granularity_list = ["weekly"]

def get_dump_count(path_str):
    for granularity in granularity_list:
        for date_str in DATE_GRANULARITY_MAPPINGLIST[granularity]:
            spark.read.parquet(path_str.format(granularity=granularity, date=date_str[:-3])).filter("date = '{date}'".format(date=date_str)).createOrReplaceTempView("test")
            df = spark.sql("select count(1) from test where kpi=1 and estimate is not null and estimate <> 0").collect()
            
            print str(granularity)+str(" ")+str(date_str)+str(" ")+str(df[0][0])

def tmp_check():
    tmp_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v2/fact/granularity=monthly/month=2014-12/"
    spark.read.parquet(tmp_path).createOrReplaceTempView("test")
    spark.sql("select distinct store_id from test order by store_id desc limit 100").show(100, False)
get_dump_count(segment_by_product_sql)

In [0]:
 

import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from pyspark.sql import Row
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY
from pyspark.sql.types import StructType, StructField, LongType, IntegerType, DoubleType, ShortType

PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)

seg_by_product_country = """
select distinct store_id from plproxy.execute_select_nestloop($proxy$ 
    select distinct store_id
    from au.app_{granularity}
    WHERE date='{date}'
    and kpi=1
    and estimate is not null
    and estimate <> 0
$proxy$) tpl (store_id int);
"""

app_x_app_country = """
select distinct store_id from plproxy.execute_select_nestloop($proxy$ 
    select distinct store_id
    from ca.app_{granularity}
    where 
        date ='{date}'
        and kpi = 9
        and estimate is not null
        and estimate <> 0
$proxy$) tpl (store_id int);
"""
segment_by_product="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v2/fact/granularity={granularity}/month={date}"
app_x_app = "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ca_app.v2/fact/granularity={granularity}/month={date}/"

def get_plproxy_result(sql_str):
    plproxy_result = []
    result = query(PLPROXY_DSN, sql_str)
    df_data = [Row(store_id=r[0]) for r in result]
    _schema =StructType([StructField("store_id", IntegerType(), False)])
    df_plproxy = spark.createDataFrame(data=df_data, schema=_schema)
    return df_plproxy
    
def get_unified_result(unified_path, granularity, date):
    spark.read.parquet(unified_path.format(granularity=granularity, date=date[:-3])).filter("date = '{date}'".format(date=date)).createOrReplaceTempView("test")
    spark.sql("select distinct store_id from test where kpi=9 and estimate <> 0 and estimate is not null").createOrReplaceTempView("df_unified")
    
def compare():
    granularity_list = ["monthly"]
    for granularity in granularity_list:
        for date in  DATE_GRANULARITY_MAPPINGLIST[granularity]:
            df_plproxy = get_plproxy_result(app_x_app_country.format(granularity=granularity, date=date))
            get_unified_result(app_x_app, granularity, date)
            df_plproxy.createOrReplaceTempView("df_plproxy")
            df = spark.sql("select store_id from df_unified except select store_id from df_plproxy").collect()
            print granularity, date, [_d[0] for _d in df]
            # print [_d[0] for _d in df]

compare()            
